In [2]:
import cv2
import dlib
import numpy as np

def eye_aspect_ratio(eye):
    # Calculate the euclidean distances between the vertical eye landmarks
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    
    # Calculate the euclidean distance between the horizontal eye landmarks
    C = np.linalg.norm(eye[0] - eye[3])
    
    # Calculate the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    
    return ear


# Load the face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Start the video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    
    for face in faces:
        landmarks = predictor(gray, face)
        
        # Extract left and right eye landmarks
        left_eye = landmarks.parts()[36:42]
        right_eye = landmarks.parts()[42:48]
        
        # Calculate eye aspect ratio (EAR) for left eye
        left_ear = eye_aspect_ratio(left_eye)
        
        # Calculate eye aspect ratio (EAR) for right eye
        right_ear = eye_aspect_ratio(right_eye)
        
        # Average the EAR for both eyes
        ear = (left_ear + right_ear) / 2
        
        # Define a threshold for blinking detection
        threshold = 0.2
        
        if ear < threshold:
            cv2.putText(frame, "Blinking Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'dlib'

In [1]:
import cv2
import numpy as np

# Load the face cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start the video capture
cap = cv2.VideoCapture(0)

def eye_aspect_ratio(eye):
    if len(eye) != 5:
        return 0
    A = np.linalg.norm(eye[1] - eye[4])
    B = np.linalg.norm(eye[2] - eye[3])
    C = np.linalg.norm(eye[0] - eye[4])
    ear = (A + B) / (2.0 * C)
    return ear

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        
        # Detect eyes within the face region
        eyes = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml').detectMultiScale(roi_gray)
        
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(frame, (x+ex, y+ey), (x+ex+ew, y+ey+eh), (0, 255, 0), 2)
            
            # Extract left and right eye landmarks
            left_eye = [(x+ex, y+ey), (x+ex+ew//2, y+ey+eh//2)]
            right_eye = [(x+ex+ew//2, y+ey), (x+ex+ew, y+ey+eh//2)]
            
            # Calculate eye aspect ratio (EAR) for left eye
            left_ear = eye_aspect_ratio(left_eye)
            
            # Calculate eye aspect ratio (EAR) for right eye
            right_ear = eye_aspect_ratio(right_eye)
            
            # Average the EAR for both eyes
            ear = (left_ear + right_ear) / 2
            
            # Define a threshold for blinking detection
            threshold = 0.2
            
            if ear < threshold:
                cv2.putText(frame, "Blinking Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np

# Load the face cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start the video capture
cap = cv2.VideoCapture(0)

def eye_aspect_ratio(eye):
    if len(eye) != 5:
        return 0
    A = np.linalg.norm(eye[1] - eye[4])
    B = np.linalg.norm(eye[2] - eye[3])
    C = np.linalg.norm(eye[0] - eye[4])
    ear = (A + B) / (2.0 * C)
    return ear

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    blink_detected = False
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        
        # Detect eyes within the face region
        eyes = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml').detectMultiScale(roi_gray)
        
        for (ex, ey, ew, eh) in eyes:
            # cv2.rectangle(frame, (x+ex, y+ey), (x+ex+ew, y+ey+eh), (0, 255, 0), 2)
            
            # Extract left and right eye landmarks
            left_eye = [(x+ex, y+ey), (x+ex+ew//2, y+ey+eh//2)]
            right_eye = [(x+ex+ew//2, y+ey), (x+ex+ew, y+ey+eh//2)]
            
            # Calculate eye aspect ratio (EAR) for left eye
            left_ear = eye_aspect_ratio(left_eye)
            
            # Calculate eye aspect ratio (EAR) for right eye
            right_ear = eye_aspect_ratio(right_eye)
            
            # Average the EAR for both eyes
            ear = (left_ear + right_ear) / 2
            
            # Define a threshold for blinking detection
            threshold = 0.2
            
            if ear < threshold:
                blink_detected = True
    
    for (x, y, w, h) in faces:
        if blink_detected:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()